In [ ]:
# https://pystac-client.readthedocs.io/en/stable/tutorials/authentication.html

In [3]:
import base64
import pystac_client
import boto3
import botocore.auth
import botocore.awsrequest
import pystac_client
import requests

In [2]:
# encode credentials
user_name = "yellowbeard"
password = "yaarg"
userpass = f"{user_name}:{password}"
b64_userpass = base64.b64encode(userpass.encode()).decode()


# create the client
client = pystac_client.Client.open(
    url="https://planetarycomputer.microsoft.com/api/stac/v1",
    headers={
        'Authorization': f"Basic {b64_userpass}"
    }
)

# Token auth

### AWS SigV4

In [4]:
# Details regarding the private stac api
region = "us-east-1"
service_name = "execute-api"
endpoint_id = "xxxxxxxx"
deployment_stage = "dev"
stac_api_url = f"https://{endpoint_id}.{service_name}.{region}.amazonaws.com/{deployment_stage}"

# load AWS credentials
credentials = boto3.Session(region_name=region).get_credentials()
signer = botocore.auth.SigV4Auth(credentials, service_name, region)

In [5]:
def sign_request(request: requests.Request) -> requests.Request:
    """Sign the request using AWS SigV4.

    Args:
        request (requests.Request): The fully populated request to sign.

    Returns:
        requests.Request: The provided request object, with auth header added.
    """
    aws_request = botocore.awsrequest.AWSRequest(
        method=request.method,
        url=request.url,
        params=request.params,
        data=request.data,
        headers=request.headers
    )
    signer.add_auth(aws_request)
    request.headers = aws_request.headers
    return request

In [6]:
# create the client
client = pystac_client.Client.open(
    url=stac_api_url,
    request_modifier=sign_request
)

APIError: HTTPSConnectionPool(host='xxxxxxxx.execute-api.us-east-1.amazonaws.com', port=443): Max retries exceeded with url: /dev (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7fc562864e10>: Failed to resolve 'xxxxxxxx.execute-api.us-east-1.amazonaws.com' ([Errno -2] Name or service not known)"))